In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import chardet
import re
import warnings
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, Concatenate, Input, RepeatVector
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [45]:
df_exploded = pd.read_csv('data_output/df_exploded_full.csv')

In [46]:
df_exploded.shape

(1679050, 9)

In [118]:
#Conditional column: where the fourth item in the 'Sequence' column contains 's' new column return 's', when 'b' return 'b', when 'f' return 'f', and when other return 'o'

df_exploded_test = df_exploded[df_exploded['Sequence'].str.split().apply(len) > 3]
df_exploded_test['cuarto'] = df_exploded_test['Sequence'].str.split().str[3].apply(
    lambda x: x[0] if isinstance(x, str) and x[0] in ['s', 'b', 'f'] else 'o'
)
df_exploded_test['cuarto'].value_counts(normalize=True)

C:\Users\agusd\AppData\Local\Temp\ipykernel_11252\1369187152.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exploded_test['cuarto'] = df_exploded_test['Sequence'].str.split().str[3].apply(


f    0.439085
b    0.344230
o    0.119495
s    0.097190
Name: cuarto, dtype: float64

In [48]:
jugador_elegido = "Roger Federer"

df_exploded = df_exploded.dropna(subset=['Sequence'])

df_exploded_filtered = df_exploded[df_exploded['Sequence'].str.split().apply(len) > 3]
# Filter rows where the fourth item in the 'Sequence' column contains 's', 'b', or 'f'
df_exploded_filtered = df_exploded_filtered[
    df_exploded_filtered['Sequence'].str.split().str[3].str.contains(r'^[sbf]')
]

print(f"Total de filas: {df_exploded_filtered.shape[0]}")

Total de filas: 261967


In [112]:
df_exploded_filtered

,Player,Sequence,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,X,y
3,Novak Djokovic,6 f1 s1 f2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0,"[6, f1, s1]",f
23,Novak Djokovic,4 b2 b1 b3 f2 f1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,11,0,0,1.0,1.0,0-0,"[4, b2, b1]",b
24,Ben Shelton,b28 f2 s3 f1 b1 b3n@,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,11,0,0,1.0,1.0,0-0,"[b28, f2, s3]",f
53,Ben Shelton,4 f3 f1 f1 f3w@,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,25,0,0,3.0,2.0,0-0,"[4, f3, f1]",f
54,Novak Djokovic,f29 b2 f1 f1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,25,0,0,3.0,2.0,0-0,"[f29, b2, f1]",f
...,...,...,...,...,...,...,...,...,...,...,...
1679003,Luis Ayala,4 s3 s3 f3 s3 s1 f3 s1 m2 s3 m2,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,155,1,1,3.0,4.0,15-30,"[4, s3, s3]",f
1679004,Nicola Pietrangeli,b37 s3 b2 f3 b3 f3 s3 r3 b3 s3 o3*,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,155,1,1,3.0,4.0,15-30,"[b37, s3, b2]",f
1679022,Luis Ayala,6 z3 o2 s3 s1 s2 s3 l2d@,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,164,1,1,5.0,4.0,15-15,"[6, z3, o2]",s
1679023,Nicola Pietrangeli,b28 m2 s2 b3 f3 s3 s1,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,164,1,1,5.0,4.0,15-15,"[b28, m2, s2]",b


In [49]:
df_exploded_filtered['Sequence'].str.split().str[3].str[0].value_counts(normalize=True)

f    0.498673
b    0.390946
s    0.110380
Name: Sequence, dtype: float64

In [59]:
df_exploded_filtered['X'] = df_exploded_filtered['Sequence'].str.split().str[:3]
df_exploded_filtered['y'] = df_exploded_filtered['Sequence'].str.split().str[3].str[0]

df_exploded_filtered['y'].value_counts(normalize=True)

f    0.498673
b    0.390946
s    0.110380
Name: y, dtype: float64

In [121]:
#resample df_exploded_filtered so that f = 40% and s = 30% and b = 30%
from sklearn.utils import resample

# Separar las clases
f_class = df_exploded_filtered[df_exploded_filtered['y'] == 'f']
b_class = df_exploded_filtered[df_exploded_filtered['y'] == 'b']
s_class = df_exploded_filtered[df_exploded_filtered['y'] == 's']

# Número total de muestras objetivo
total_samples = len(df_exploded_filtered)

# Calcular el número de muestras para cada clase según las proporciones deseadas
f_target = int(total_samples * 0.4)
b_target = int(total_samples * 0.35)
s_target = int(total_samples * 0.25)

# Aplicar resampling (sobremuestreo o submuestreo)
f_resampled = resample(f_class, replace=True, n_samples=f_target, random_state=42)
b_resampled = resample(b_class, replace=True, n_samples=b_target, random_state=42)
s_resampled = resample(s_class, replace=True, n_samples=s_target, random_state=42)

# Combinar las clases resampleadas
df_resampled = pd.concat([f_resampled, b_resampled, s_resampled])

# Barajar el dataset para mezclar las clases
df_resampled = df_resampled.sample(frac=1, random_state=42).reset_index(drop=True)

df_exploded_filtered = df_resampled.copy() 

In [122]:
df_exploded_filtered[['Player', 'X', 'y']].head()

,Player,X,y
0,Vitas Gerulaitis,"[6, f3, s3]",s
1,Rafael Nadal,"[6, s2, f2]",b
2,Jimmy Connors,"[f38, f1, f3]",b
3,Roger Federer,"[f38, b2, f1]",b
4,Stefan Edberg,"[s38, s2, f2]",s


In [123]:
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

n_steps = 3

X = df_exploded_filtered['X']
players = df_exploded_filtered['Player']
y = df_exploded_filtered['y']

# Tokenizador para las secuencias
tokenizer = Tokenizer()
tokenizer.fit_on_texts([' '.join(seq) for seq in X])
X_encoded = tokenizer.texts_to_sequences([' '.join(seq) for seq in X])
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_encoded, maxlen=n_steps)

# Codificar jugadores
player_encoder = LabelEncoder()
players_encoded = player_encoder.fit_transform(players)

# Codificar etiquetas (y)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)


In [124]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50

# Input 1: Secuencia
seq_input = Input(shape=(n_steps,), name='sequence_input')
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(seq_input)
lstm = LSTM(64)(embedding)

# Input 2: Jugador
player_input = Input(shape=(1,), name='player_input')
player_embedding = Embedding(input_dim=len(player_encoder.classes_), output_dim=5)(player_input)
player_flattened = tf.keras.layers.Flatten()(player_embedding)

# Combinar ambas entradas
combined = Concatenate()([lstm, player_flattened])

# Salida
output = Dense(y_categorical.shape[1], activation='softmax', name='output')(combined)

# Crear modelo
model = Model(inputs=[seq_input, player_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sequence_input      │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ player_input        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 3, 50)     │      7,350 │ sequence_input[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 5)      │      3,965 │ player_input[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     29,440 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 5)         │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 69)        │          0 │ lstm_1[0][0],     │
│ (Concatenate)       │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 3)         │        210 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 40,965 (160.02 KB)

 Trainable params: 40,965 (160.02 KB)

 Non-trainable params: 0 (0.00 B)

None


In [125]:
# Dividir en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, players_train, players_test = train_test_split(
    X_padded, y_categorical, players_encoded, test_size=0.2, random_state=42
)

# Entrenar el modelo
history = model.fit(
    [X_train, players_train],
    y_train,
    validation_data=([X_test, players_test], y_test),
    epochs=20,
    batch_size=32
)


Epoch 1/20


C:\Users\agusd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['sequence_input', 'player_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


6550/6550 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.4627 - loss: 1.0264 - val_accuracy: 0.4796 - val_loss: 1.0059
Epoch 2/20
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.4827 - loss: 1.0033 - val_accuracy: 0.4838 - val_loss: 1.0039
Epoch 3/20
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.4850 - loss: 1.0009 - val_accuracy: 0.4855 - val_loss: 1.0024
Epoch 4/20
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.4900 - loss: 0.9972 - val_accuracy: 0.4834 - val_loss: 1.0029
Epoch 5/20
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.4900 - loss: 0.9959 - val_accuracy: 0.4860 - val_loss: 1.0024
Epoch 6/20
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.4894 - loss: 0.9960 - val_accuracy: 0.4854 - val_loss: 1.0017
Epoch 7/20
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.4905 - loss: 0.9954 - val_accuracy: 0.4872 - val_loss: 1.0018
Epoch 8/20
6550/6550 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.4918 - loss: 0.9932 - val

In [126]:
# Predecir el próximo golpe
def predict_next_shot(sequence, player):
    # Preprocesar entrada
    seq_encoded = tokenizer.texts_to_sequences([sequence])
    seq_padded = tf.keras.preprocessing.sequence.pad_sequences(seq_encoded, maxlen=n_steps)
    player_encoded = player_encoder.transform([player])
    
    # Hacer predicción
    prediction = model.predict([seq_padded, player_encoded])[0]  # Vector de probabilidades para la fila
    
    # Mapear índices a las clases
    class_probabilities = {label_encoder.inverse_transform([i])[0]: prob for i, prob in enumerate(prediction)}
    return class_probabilities

# Evaluar en el conjunto de prueba
loss, accuracy = model.evaluate([X_test, players_test], y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

1638/1638 ━━━━━━━━━━━━━━━━━━━━ 1s 824us/step - accuracy: 0.4917 - loss: 0.9970
Test Loss: 0.9963498711585999, Test Accuracy: 0.4921458959579468


In [127]:
secuencia = ['f2', 's2', 'f3']

print(predict_next_shot(secuencia, "Roger Federer"))
print(predict_next_shot(secuencia, "Novak Djokovic"))
print(predict_next_shot(secuencia, "Rafael Nadal"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
{'b': 0.15614006, 'f': 0.15875413, 's': 0.68510586}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


C:\Users\agusd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['sequence_input', 'player_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


{'b': 0.30139023, 'f': 0.23746009, 's': 0.4611497}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
{'b': 0.23236082, 'f': 0.20397472, 's': 0.5636645}
